# vision model:
models that extract data from images.

input1 : image
input2 : prompt -"Where is the photographer resting?"

In [ ]:
image -> contrastive vision encoder -> linear projection ->embeddingImage

prompt -> tokenizer( sentencePiece) ->embedddingPrompt

=> concatenated embedding (embeddingImage + embedddingPrompt) -> transformer decoder -> response

Each input image will be split into  blocks  of pixels creating a grid; then this grid will be converted into an embedding. This embedding is a vector of fixed size.. that will be concenated with tokens' embeddings of  input prompt text i.e. each token will have an embedding just as each block of pixels will have an embedding. then this concatenated vetor will be sent to transformer decoder.

![alt text](vision_lang_model_01.png)
